# RTT and Window size effects on TCP
<i>Adapted for use with FABRIC from [Effect of RTT and Window Size on TCP](https://www.cs.unc.edu/Research/geni/geniEdu/04-TcpDelayWinSize.html)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)


<br><img src='https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/RTT%20and%20Window%20Size%20on%20TCP/figures/RTTTop.png?raw=true'><br>
In this notebook module you will be able to experiment how RTT and TCP window size affect the throughput. You will learn how to adjust the RTT by adjusting time delay, and you will experiment with the window size using the iperf commands.
This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.
#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = 'RttWindowTCP'
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2. Run Experiment
### 2.1 Adjusting the MTU (Maximum Transfer Unit)
In this step we will setup the MTU to use later in the tutorial.
1. Login to the each node and keep a terminal open on each node.
2. Run `ifconfig` on the terminal Server
   <br> You should take a note of the interface that contains the server's assigned ip address (10.1.1.1), The interface is of the form ```enp#s0``` where '#' is some number. This will be the interface that is able to connect the server and the client.
3. Since this is a GRE tunnel, you can adjust the MTU size for the interfaces. This can be done with the following command:
	<br> `sudo ifconfig <interface name> mtu #`
The MTU can be adjusted as needed, but for this experiment we will use `# = 1400`
4. Repeat steps 2 and 3 on the client node. Remember that it is likely that the interfaces are the same, but it is not guaranteed



### 2.2 Adding and Removing Delay
<img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/RTT%20and%20Window%20Size%20on%20TCP/figures/RTT.png?raw=true">

In this step we will find how to change the delay of transfer.
1. On the Client terminal run the following steps
2. First we need to know what the approximate base RTT (Round Trip Time) between the nodes is. We are able to find this information using the ping tool: `ping <Server ip addr> -c 10`
We will ping 10 times to find an average RTT in the report. Alternatively you can get an estimate by looking at the individual pings.
3. We can use a Linux tool called tc (traffic control) to add a fixed amount of delay to the packets leaving the interface.
	<br> In this case the delay will be 100ms. Use the following command on the Server terminal:
	<br>`sudo tc qdisc add dev <interface name> root netem delay 100ms`
	<br> The command translates to: adjust tc queuing discipline (qdisc) by adding (add) a network emulation (netem) delay of 100 milliseconds to the device at the root.
4. If we try running the ping again `ping <Server ip addr> -c 10`
	<br> we should now see that the RTT average is 100ms longer
5. We are also able to remove the delay using `sudo tc qdisc del dev <interface name> root`
6. Pinging the Server should now be the same as it was before adding the delay `ping <Client ip addr> -c 10`


### 2.3 Adjusting The TCP window size
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/RTT%20and%20Window%20Size%20on%20TCP/figures/WindowSize.png?raw=true"><br>
In this step, we will learn how to adjust the window size and understand its impact on performance. We can observe that with relatively larger window sizes, we can send more information and occupy more bandwidth compared to relatively small window sizes. Larger window sizes allow for less wait time between network acknowledgment communications and facilitate the transmission of more information. However, it's important to note that there can be trade-offs if your window size is too large, such as potential packet loss.
1. iperf is a tool for measuring TCP and UDP bandwidth performance. This tool allows us to change the window size. In the Server Terminal run the following command: `iperf -s`
	<br> This node will run iperf in server mode "-s" and it allows it to receive iperf traffic
2. On the Client terminal, start the ipref client using `iperf -c <server ip addr> -t 10`
	<br> This command starts iperf in client mode "-c" and opens a TCP connection to the server. The client will begin sending packets using the default window size. The client will stop after 10 seconds "-t 10".
3. Once the Client is done take note of the average bandwidth and use "Ctrl-c" in the Server terminal to kill the server
4. Now we can repeat the procedure, but we will limit the window size on both the client and the server using the following command: `iperf -s -w 2KB`
	<br> Note that Linux automatically doubles the window size set by iperf.
5. In the client terminal type: `iperf -c <server ip addr> -t 10 -w 2KB` to start the connection
	<br> Once the client is done you can compare the average bandwidth value and notice that it is significantly less than when we used the default window size.

## 3.  Assignment
### Part 1: Setup
For this assignment, you will utilize the total traffic graphs to plot and compare the results of adjusting the RTT and Window size parameters. Run experiments for every combination of the two window sizes (4KB and 32KB) and two delay amounts (50ms and 250ms) and take a screenshot of the graphs from the Client node after each experiment has completed. You should have 4 pictures in total (e.g. one image for each of the two nodes for each of the following combinations: 4KB and 50ms, 4KB and 250ms, 32KB and 50ms, and 32KB and 250ms).
#### How to make a graph
To make a graph you will need 2 windows on the client and one on the server.
1. Set up the intended time delay using the command you learned from section 2.2
2. Start the server side iperf (Do not start the client).
3. On one of the windows to the client (A) you will start the pre-loaded script with the command ```RT-output.sh```
4. On the second Terminal start the client connection to server via iperf.
5. Once completed on Terminal B hit ```Ctrl-c``` on Client Terminal A.
6. Run the code block bellow, then open (or close and then re-open) the '.sgv' file to look at your graph.
*Note: each time you make a graph the old one will be deleted. You can prevent this by renaming the file each time*
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/RTT%20and%20Window%20Size%20on%20TCP/figures/Gsteps.png?raw=true">


In [ ]:
# Download From Client Node

try:
    A = slice.get_node('Client')
    A.execute("Rscript RT-data-analysis.R  ")
    A.download_file("Traffic-RT.svg","Traffic-RT.svg")
except Exception as e:
    print(f"Fail: {e}")

### Part 2: Analysis
You should have noticed a large difference in average throughput between the 4KB and 250ms case and the 32KB and 50ms case.

Justify this difference in your own words and by referencing the TCP equation given in your notes.

*Note: If you actually work out the math with the TCP equation, you should use the total delay of your system, not just the delay you added.*


## Cleanup Resources
Once you have completed the assignment shut down the slice

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")